# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

09:42:58.262592 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
09:42:58.263659 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
09:42:58.263692 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
09:42:58.267809 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
09:42:58.292429 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fd43f246b50>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{message}{CRESET}")
        
#cprint = lambda tag, message: None    # Uncomment this to disable Python logging
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
{"c": client_tiface, "s": server_tiface}

{'c': <__main__.LoggingTransportInterface at 0x7fd43f0f2d70>,
 's': <__main__.LoggingTransportInterface at 0x7fd43f0f2f50>}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7fd43f12cfb0>,
 's': <FilterInterface at 0x7fd43f12e180>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
09:42:58.376340 [INFO ] - [F] About to send pings to servers: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server (both our local and Chococat) as ping endpoints

In [10]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

09:42:58.575264 [INFO ] - [F] About to send pings to servers: []
09:42:58.775034 [INFO ] - [F] About to send pings to servers: []


'157.230.134.224'

In [11]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }

In [12]:
await client_fiface.command_response(filter_cmd)

09:42:58.970993 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }
09:42:58.974978 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]


Accepted

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

In [13]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

09:42:59.174693 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:42:59.174739 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:42:59.174763 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:42:59.374974 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:42:59.374996 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:42:59.374999 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:42:59.575388 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:42:59.575410 [I

[]

In [14]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

Sleeping again....

In [15]:
time.sleep(10.0)
client_notifications = client_fiface.get_notifications()
client_notifications

09:43:02.175838 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:02.175891 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:02.175900 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:02.375080 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:02.375105 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:02.375109 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:02.575220 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:02.575261 [I

09:43:06.174948 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:06.175027 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:06.175033 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:06.374649 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:06.374676 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:06.374682 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:06.575628 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:06.575655 [I

09:43:10.174949 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:10.175003 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:10.175012 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:10.375256 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:10.375283 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 18128269235191945562!
09:43:10.375290 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 1762832923625281424!
09:43:10.575191 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
09:43:10.575220 [I

[]

In [16]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

LTI(client)138 skipped 137: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 18128269235191945562 } }] }
09:43:12.034873 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 18128269235191945562 } }] }
09:43:12.034968 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 18128269235191945562 } }


Accepted

09:43:12.035803 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
09:43:12.035821 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]


In [17]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

### Now shutdown

In [ ]:
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))

LTI(client)139: got transport notifications: []
09:43:12.058832 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:51131), took packet GetStatus { ping: PingPong { nonce: 18128269235191945562 } }
09:43:12.058861 [INFO ] - [F] Sending Status packet PingPong { nonce: 18128269235191945562 } back to client Endpoint(127.0.0.1:51131)
LTI(client)140: got transport_rsp back from Transport layer: TransportRsp::Accepted
09:43:12.060581 [TRACE] - [F<-A,C] New command: Shutdown { graceful: true }
09:43:12.060604 [INFO ] - [F] shutting down
LTI(server)440 skipped 439: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:51131), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 0ns }], packets: [Status { pong: PingPong { nonce: 18128269235191945562 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }] }
LTI(client)141 skipped 0: received transport_cmd SendPackets { endpoint: E